In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import pandas as pd

df = pd.read_csv("for_gensim.csv")

df.head()



,Unnamed: 0,Date,Time,formtype,Improve,Imp1,Imp2,ImpCrit,CommentCoderImp,TweetImp,Best,Best1,Best2,BestCrit,Division2,Directorate2,Location,Keep_Improve,comment,word_count
0,1,2009-04-01,1,adult,the time it took from the gps referral to gett...,NaN,NaN,NaN,NaN,NaN,X was an excellent therapist who helped me a lot.,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,The time it took from the G.P's referral to ge...,1,20
1,2,2009-04-01,1,adult,an advocate to support me when im ill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,An advocate to support me when I'm ill,2,8
2,3,2009-04-01,1,adult,at this time i am unsure at to what could be i...,NaN,NaN,NaN,NaN,NaN,Meeting the team who are caring for my wife in...,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,At this time I am unsure at to what could be i...,3,12
3,4,2009-04-01,1,adult,at times the service can be a bit slow eg 3 we...,NaN,NaN,NaN,NaN,NaN,Felt very reassured and supported at all times...,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,"At times the service can be a bit slow, e.g. 3...",4,15
4,5,2009-04-01,1,adult,being told what is going on and helped to unde...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,Being told what is going on and helped to unde...,5,13


In [2]:
# set up dataframe

df['Date'] = pd.to_datetime(df['Date'])

df["tokens"] = df["Improve"].apply(lambda x: x.split()).tolist()

print(df.head())


   Unnamed: 0       Date  Time formtype  \
0           1 2009-04-01     1    adult   
1           2 2009-04-01     1    adult   
2           3 2009-04-01     1    adult   
3           4 2009-04-01     1    adult   
4           5 2009-04-01     1    adult   

                                             Improve  Imp1  Imp2  ImpCrit  \
0  the time it took from the gps referral to gett...   NaN   NaN      NaN   
1              an advocate to support me when im ill   NaN   NaN      NaN   
2  at this time i am unsure at to what could be i...   NaN   NaN      NaN   
3  at times the service can be a bit slow eg 3 we...   NaN   NaN      NaN   
4  being told what is going on and helped to unde...   NaN   NaN      NaN   

  CommentCoderImp  TweetImp  \
0             NaN       NaN   
1             NaN       NaN   
2             NaN       NaN   
3             NaN       NaN   
4             NaN       NaN   

                         ...                         Best1  Best2  BestCrit  \
0           

In [4]:
%%time

# fit the model

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df["tokens"])]

# model = gensim.models.Doc2Vec(documents, dm = 0, alpha = 0.025, vector_size = 20, 
#                              min_alpha = 0.025, min_count = 0, epoch = 20)

# fit a model with a decay in the learning rate

model = Doc2Vec(alpha=0.025, min_alpha=0.025, vec_size = 100, workers = 4)  # use fixed learning rate
model.build_vocab(documents)
for epoch in range(20):
    model.train(documents, total_examples=model.corpus_count, epochs = 100)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay
                              
print("No output here")                              


No output here
CPU times: user 3min 41s, sys: 38.2 s, total: 4min 19s
Wall time: 2min 33s


In [5]:
%%time

def similar_documents(comment):
    store_similar = []
    tokens = comment
    new_vector = model.infer_vector(tokens)
    sims = model.docvecs.most_similar([new_vector])
    res_list = [x[0] for x in sims]
    store_similar.append(res_list)
    return store_similar
    
df["similar_vector"] = df["Improve"].apply(lambda x: similar_documents(x))


CPU times: user 7min 7s, sys: 2.46 s, total: 7min 10s
Wall time: 1min 47s


In [21]:
similar_documents('more biscuits')

[[2264, 36040, 7984, 14997, 11359, 34715, 8550, 9438, 21384, 12277]]

In [27]:
df['Improve'][2263: 2266].values

array(['was expecting letter detailing more appointments from care coordinator which didnt materialise which caused stress leading to crisis',
       'better wheelchair accessibility grab rails in toilets and bathrooms bedroom laid out better due to wheelchair',
       'more communication between staff and patients'], dtype=object)

In [29]:
df['Improve'][7983: 7986].values

array(['the communication between officers and staff of healthcare is not existant',
       'faster times for bloods etc', 'better care more manners'],
      dtype=object)

In [30]:
df['Improve'][36039: 36042].values

array(['more staff less agency who some times burden the hard working nurses',
       'answer emergency a bit quicker', 'control the noise ie tvstaff'],
      dtype=object)

In [6]:
df.head()

,Unnamed: 0,Date,Time,formtype,Improve,Imp1,Imp2,ImpCrit,CommentCoderImp,TweetImp,...,Best2,BestCrit,Division2,Directorate2,Location,Keep_Improve,comment,word_count,tokens,similar_vector
0,1,2009-04-01,1,adult,the time it took from the gps referral to gett...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,The time it took from the G.P's referral to ge...,1,20,"[the, time, it, took, from, the, gps, referral...","[[25377, 9423, 23127, 14867, 6314, 23749, 2618..."
1,2,2009-04-01,1,adult,an advocate to support me when im ill,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,An advocate to support me when I'm ill,2,8,"[an, advocate, to, support, me, when, im, ill]","[[17317, 3226, 27480, 10314, 7064, 22650, 2378..."
2,3,2009-04-01,1,adult,at this time i am unsure at to what could be i...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,At this time I am unsure at to what could be i...,3,12,"[at, this, time, i, am, unsure, at, to, what, ...","[[28291, 6071, 30523, 7373, 34510, 8746, 14205..."
3,4,2009-04-01,1,adult,at times the service can be a bit slow eg 3 we...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,"At times the service can be a bit slow, e.g. 3...",4,15,"[at, times, the, service, can, be, a, bit, slo...","[[9423, 23127, 8746, 35234, 4045, 26187, 25377..."
4,5,2009-04-01,1,adult,being told what is going on and helped to unde...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,Being told what is going on and helped to unde...,5,13,"[being, told, what, is, going, on, and, helped...","[[15946, 2890, 16637, 8035, 9741, 1049, 7124, ..."


In [14]:
# save

df.to_csv(path_or_buf = "/home/chris/Nextcloud/DataScienceAccelerator/shiny_doc2vec/document_results.csv", index = True)